# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [11]:
# Write your code below.
import os
os.chdir('/Users/danicaleung/DSI_Repo_Assignments/production/05_src')
os.getcwd()
%load_ext dotenv
%dotenv

In [12]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [14]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [16]:
# Write your code below.
dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
                lambda x: x.assign(Close_lag = x['Close'].shift(1))))
                
                
dd_feat = (dd_feat.groupby('ticker', group_keys = False).apply(
                lambda x: x.assign(Adj_Close_lag = x['Adj Close'].shift(1)
            ).assign(
                returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
            ).assign(
                hi_lo_range = lambda x: (x['High'] - x['Low'])
            )))

dd_feat

/opt/miniconda3/envs/dsi_participant/lib/python3.9/site-packages/dask/dataframe/core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
/var/folders/vk/7f5v13993c58cswjwb8_rwvm0000gq/T/ipykernel_11414/2000549690.py:4: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
/var/folders/vk/7f5v13993c58cswjwb8_rwvm0000gq/T/ipykernel_11414/2000549690.py:8: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_feat.groupby('

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range
npartitions=11207,,,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,int64,string,string,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [18]:
pd_mean = dd_feat.compute()
pd_mean['rolling_average'] = pd_mean['returns'].rolling(10).mean()
pd_mean

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,rolling_average
ticker,,,,,,,,,,,,,,,
HUM,2012-01-03,89.209999,89.500000,87.839996,88.330002,79.585213,1287300,Health Care,Managed Health Care,2012,NaN,NaN,NaN,1.660004,NaN
HUM,2012-01-04,88.339996,90.389999,87.779999,89.930000,81.026817,1972000,Health Care,Managed Health Care,2012,88.330002,79.585213,0.018114,2.610001,NaN
HUM,2012-01-05,89.430000,91.760002,89.199997,91.680000,82.603592,1954400,Health Care,Managed Health Care,2012,89.930000,81.026817,0.019460,2.560005,NaN
HUM,2012-01-06,91.980003,94.849998,91.750000,92.250000,83.117165,2809800,Health Care,Managed Health Care,2012,91.680000,82.603592,0.006217,3.099998,NaN
HUM,2012-01-09,92.980003,93.449997,91.779999,93.250000,84.018173,1512900,Health Care,Managed Health Care,2012,92.250000,83.117165,0.010840,1.669998,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2012-12-24,19.700001,20.070000,19.420000,20.010000,15.593549,1860200,Information Technology,Communications Equipment,2012,20.170000,15.718231,-0.007932,0.650000,0.003798
JNPR,2012-12-26,20.010000,20.400000,19.770000,19.889999,15.500036,2943200,Information Technology,Communications Equipment,2012,20.010000,15.593549,-0.005997,0.629999,0.003769
JNPR,2012-12-27,19.930000,19.950001,19.430000,19.790001,15.422103,4461300,Information Technology,Communications Equipment,2012,19.889999,15.500036,-0.005028,0.520000,0.004309


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Yes it was necessary to convert to pandas. By computing first and then calculating the moving average, the run time was about 2 minutes. If the average was calculated first and then converted to pandas, the run time was more than twice as long at 4 minutes 55 seconds. For efficiency and to save on computing costs, it was better to do the calculation in pandas rather than in Dask.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.